# 2.5.2 Validating a linear regression

In [ ]:
import math
import warnings

from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf

# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format

# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

In [75]:
data = pd.read_excel('C:/Personal/09142640/Downloads/Crimes_new_york_by_city_2014.xls', sheet_name=0, header=4)
data.columns.values[8]='Property_Crime'
data.head()

,City,Population,Violent crime,Murder and nonnegligent manslaughter,Rape (revised definition)1,Rape (legacy definition)2,Robbery,Aggravated assault,Property_Crime,Burglary,Larceny- theft,Motor vehicle theft,Arson3
0,Adams Village,1851.000,0.000,0.000,nan,0.000,0.000,0.000,11.000,1.000,10.000,0.000,0.000
1,Addison Town and Village,2568.000,2.000,0.000,nan,0.000,1.000,1.000,49.000,1.000,47.000,1.000,0.000
2,Afton Village4,820.000,0.000,0.000,0.000,nan,0.000,0.000,1.000,0.000,1.000,0.000,0.000
3,Akron Village,2842.000,1.000,0.000,nan,0.000,0.000,1.000,17.000,0.000,17.000,0.000,0.000
4,Albany4,98595.000,802.000,8.000,54.000,nan,237.000,503.000,3888.000,683.000,3083.000,122.000,12.000


In [76]:
data = data.drop(data[data.Population > 15000].index)

In [77]:
data['Sq_population']=data['Population']*data['Population']

In [78]:
data['Burglary_Flag']=np.where((data['Burglary']>0),1,0)

In [80]:
data['Robbery_Flag']=np.where((data['Robbery']>0),1,0)


Propery Crime=\alpha + \beta_Burglary+ \beta_2Population + \beta_3Robbery+\beta_4Population**2

In [92]:
regr = linear_model.LinearRegression()
#features=data[['Population','Burglary','Larceny-theft','Motor vehicle theft','Property_Crime']].dropna()
Y = data[['Property_Crime']]
X = data[['Burglary','Population','Robbery','Sq_population','Burglary_Flag','Robbery_Flag']]
X=X.fillna(0)
Y=Y.fillna(0)
regr.fit(X, Y)
# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))



Coefficients: 
 [[ 4.97371873e+00 -2.47077325e-03 -4.08907146e-01  4.86597634e-07
  -3.52958329e-01  2.00670218e+01]]

Intercept: 
 [5.05865105]

R-squared:
0.7975651978046295


The model shows that outcome Property crime is predicted by the features Burglary,Population,Robbery,Sq_population,Burglary_Flag,Robbery_Flag explains 79.7% of the variance in Property crime

In [93]:
correlation_matrix = X.corr()
display(correlation_matrix)

,Burglary,Population,Robbery,Sq_population,Burglary_Flag,Robbery_Flag
Burglary,1.000,0.592,0.698,0.592,0.305,0.518
Population,0.592,1.000,0.492,0.961,0.328,0.523
Robbery,0.698,0.492,1.000,0.501,0.178,0.569
Sq_population,0.592,0.961,0.501,1.000,0.236,0.472
Burglary_Flag,0.305,0.328,0.178,0.236,1.000,0.285
Robbery_Flag,0.518,0.523,0.569,0.472,0.285,1.000


Test the Model

In [94]:

data = data[['Property_Crime', 'Burglary','Population','Robbery','Sq_population','Burglary_Flag','Robbery_Flag']]
linear_formula = 'Property_Crime~Burglary+Population+Robbery+Sq_population+Burglary_Flag+Robbery_Flag'
# Fit the model to our data using the formula.
#lm = smf.ols(formula=linear_formula, data=data1).fit()

In [67]:
import statsmodels.formula.api as smf
from sklearn.linear_model import LinearRegression 
from scipy import stats
lm = smf.ols(formula=linear_formula,data=data).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:         Property_Crime   R-squared:                       0.764
Model:                            OLS   Adj. R-squared:                  0.758
Method:                 Least Squares   F-statistic:                     128.7
Date:                Fri, 14 Dec 2018   Prob (F-statistic):           8.34e-72
Time:                        11:09:43   Log-Likelihood:                -1356.0
No. Observations:                 245   AIC:                             2726.
Df Residuals:                     238   BIC:                             2751.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept         9.9418     14.885      0.668

Here we notice that that all the features execept Robbery_Flag and Burglary have p values less than 0.05.So Dropping Burglary and Robbery_flag will not significantly impact the R^2 value

Revised Model

In [97]:
# Reinstantiate and fit.
regr = linear_model.LinearRegression()
Y_test = data['Property_Crime']
X_test= data[['Population', 'Sq_population']]
X=X_test.fillna(0)
Y=Y_test.fillna(0)
regr.fit(X, Y)

# Inspect the results.
print('\nCoefficients: \n', regr.coef_)
print('\nIntercept: \n', regr.intercept_)
print('\nR-squared:')
print(regr.score(X, Y))

# Reorganize data and remove unwanted parameters
data = data[['Property_Crime', 'Population', 'Sq_population']]

# Update interaction
linear_formula = 'Property_Crime ~ Population+Sq_population'

# Run model and print results
lm_revised = smf.ols(formula=linear_formula, data=data).fit()
print(lm_revised.summary())


Coefficients: 
 [6.96723686e-03 9.45385250e-07]

Intercept: 
 10.440258716933315

R-squared:
0.39322751350285456
                            OLS Regression Results                            
Dep. Variable:         Property_Crime   R-squared:                       0.395
Model:                            OLS   Adj. R-squared:                  0.390
Method:                 Least Squares   F-statistic:                     87.36
Date:                Fri, 14 Dec 2018   Prob (F-statistic):           6.13e-30
Time:                        11:50:08   Log-Likelihood:                -1608.0
No. Observations:                 271   AIC:                             3222.
Df Residuals:                     268   BIC:                             3233.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
--------------

Here in the revised model, we notice that the F statistic value is quite high and R-squared is below 50%.Also, we are seeing that there are stroing multicolinaerity in the parameters

Conclusion:I will chose the revised model ,the reason being there are less features or components in this model which can explain the model instead of model which is having more feature and is not able to explain the model.Also, we are having p values less than 0.05% for the all features that means the features can be used in the model